In [71]:
import numpy as np
import utils
import matplotlib.pyplot as plt
from scipy.optimize import fmin_l_bfgs_b
from os.path import join

### Class for epsilon gredy method

In [72]:
class EpsilonGreedy:
    def __init__(self, epsilon, datapath):
        self.util = utils.Util(datapath)
        self.epsilon = epsilon
        self.recommended_song_ids = []
        self.recommended_song_candidate = 0

    def exploration_exploitation(self):
        if len(self.recommended_song_ids) == 0 or self.epsilon > np.random.rand():
            song_id = np.random.randint(self.util.get_number_of_songs())  # random choice
        else:
            song_id = self.recommended_song_candidate  # greedy choice
        self.recommended_song_ids.append(song_id)
        self.util.add_recommendation(song_id)
        return song_id

    def feedback(self, rating):
        self.util.add_rating(rating)
        t = self.util.get_all_times()
        x = self.util.get_all_features()
        theta, s = self.calculate_theta_s()
        self.recommended_song_candidate = np.argmax(theta.T.dot(x) * (1 - np.exp(-t / s)))
        self.calculate_cumulative_regret(theta, s) 

    def calculate_cumulative_regret(self, theta, s):
        y = self.util.get_ratings()
        t = self.util.get_history_times()
        x = self.util.get_features_of_history()
        y_model = theta.T.dot(x) * (1 - np.exp(-t / s))
        print('expected rating: ', y_model[-1])
        self.util.add_expected_rating(y_model[-1])
        self.cumulative_regret = np.average(y - y_model)

    def calculate_theta_s(self):
        initial_values = np.zeros(self.util.get_number_of_features() + 1)
        initial_values[-1] = 1
        x = self.util.get_features_of_history()
        y = self.util.get_ratings()
        t = self.util.get_history_times()
        position, _, _ = fmin_l_bfgs_b(self.optimization, x0=initial_values, args=(x, y, t), approx_grad=True)
        theta = position[:-1]
        s = position[-1]
        return theta, s     

    @staticmethod
    def optimization(params, *args):
        x = args[0]
        y = args[1]
        t = args[2]
        theta = params[:-1]
        s = params[-1]
        y_model = theta.T.dot(x) * (1 - np.exp(-t / s))
        error = y - y_model
        return sum(error ** 2)

### Calculate ratings

In [73]:
def calculate_rating(theta, s, song_features, song_times):
    return theta.T.dot(song_features) * (1 - np.exp(-song_times / s)) * 10

### Create a simulation to test the performance of epsilon greedy algorithm 

In [74]:
def generate_simulation(song_names, method, theta, s, length, datapath):
    if method == 'Random':
        model = EpsilonGreedy(1.0, datapath) #only exploration
    else:
        method == 'Greedy'
        model = EpsilonGreedy(0.2, datapath) #greedy choice

    for i in range(length):
        recommended_song = model.exploration_exploitation()
        print("Recommended song: ", i, recommended_song, ' '.join(song_names.iloc[recommended_song]))
        features, times = model.util.get_features_and_times_of_song(recommended_song)
        rating = calculate_rating(theta, s, features, times)
        print('Rating: ', rating)
        model.feedback(rating)

    cum_regret = model.util.get_cumulative_regret()
    fig1 = plt.figure(1, figsize=(10, 5))
    plt.plot(cum_regret)
    plt.title("Cumulative Regret")
    fig1.savefig(join(method + "_regret.png"))
    plt.close(fig1)

    cumulative_average_rating = model.util.get_cumulative_average_rating()
    fig2 = plt.figure(1, figsize=(10, 5))
    plt.plot(cumulative_average_rating)
    plt.title("Cumulative Average Rating")
    fig2.savefig(join(method + "_rating.png") )
    plt.close(fig2)

### Create a function to run the simulation 

In [75]:
def run_simulation():
    filepath = "songs_with_features.csv"
    np.random.seed(12)
    data, song_names = utils.get_data(filepath)
    theta = np.random.random(data.shape[1])
    s = 10
    length = 30
    methods = ["Random","Greedy"] 
    for method in methods:
        print("Starting simulation for model", method)
        print("======================================")
        generate_simulation(song_names, method, theta, s, length, filepath)
        print("End of  simulation for model", method)
        print("======================================")

### Run the simulation 

In [76]:
run_simulation()

Starting simulation for model Random
Recommended song:  0 6084 Thievery Corporation Retaliation Suite
Rating:  2.8696218578097197
expected rating:  2.8696214224436805
Recommended song:  1 1395 Cowboy Junkies Working On A Building
Rating:  2.5554330179869753
expected rating:  2.5554331648950814
Recommended song:  2 3691 Matt Costa Mr. Pitiful
Rating:  2.716355107154931
expected rating:  2.716361202527964
Recommended song:  3 3270 Lene Marlin Another Day
Rating:  2.2303519419278555
expected rating:  2.230382171396064
Recommended song:  4 6300 Treasure Fingers Cross The Dancefloor
Rating:  2.7467787840979687
expected rating:  2.7467581297505768
Recommended song:  5 4451 Rosie Thomas Much Farther To Go
Rating:  2.6878819914960217
expected rating:  2.6876841430409284
Recommended song:  6 4587 Seven Letters The Fit
Rating:  3.2294783333665023
expected rating:  3.2294885856294893
Recommended song:  7 2153 Freda Payne Band Of Gold
Rating:  2.231372538401835
expected rating:  2.2316352164744613